In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('C:/Users/au723200/FEPCEU')

In [3]:
from tqdm import tqdm

In [4]:
from enhanced_experiment import EnhancedTimeSeriesExperiment
from models.arima import Arima
from models.baseline import Baseline
from models.lgbm import LGBM
from models.neuralproph import Neuralprophet
from models.timegpt import TimeGPT

from data.data_retriever import Dataretreiver

Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [5]:
def kmeans():
    # Define models
    models_kmeans = {
        'BASELINE': Baseline(),
        'LGBM': LGBM(),
        #'ARIMA': Arima(order=[1,0,1], seasonal_order=[1,1,1,24]),
        'NEURALPROPHET': Neuralprophet(),
        'TimeGPT': TimeGPT()

    }
    print("PCA_KMEANS STARTED")
    for i in tqdm(range(28, 40)):
        data = Dataretreiver(reduce='pca_kmeans', debug=False, reduction_num_cols=i)
        df = data.combined
        # Initialize the experiment
        experiment = EnhancedTimeSeriesExperiment(
            models=models_kmeans,
            target_column='price',
            forecast_horizon=192,
            n_splits=3,
            step_size=192 # 8 days are skipped between each forecast.
        )
        # Run the experiments with these custom combinations
        experiment.run_feature_group_experiments(
            df=df,
            add_all_columns=True,
            add_base_columns=False,
        )
        results_df_MAE = experiment.summarize_feature_group_results(metric='MAE')
        results_df_MSE = experiment.summarize_feature_group_results(metric='RMSE')
        results_df_time = experiment.summarize_feature_group_results(metric='elapsed_time')
        print('='*50)
        print(f"Iterataion {i} results for pca_kmeans")
        print("MAE:")
        print(results_df_MAE)
        print("RMSE")
        print(results_df_MSE)
        print("Elapsed time")
        print(results_df_time)
        print('='*50)
        experiment.save_feature_group_results_to_csv(f"../results/opt_num_areas/pca_kmeans_{i}.csv")

In [6]:
def pure():
    print("PCA_PURE STARTED")
    # Define models
    models_pure = {
        'BASELINE': Baseline(),
        'LGBM': LGBM(),
        #'ARIMA': Arima(order=[1,0,1], seasonal_order=[1,1,1,24]),
        'NEURALPROPHET': Neuralprophet(),
        'TimeGPT': TimeGPT()

    }
    for i in tqdm(range(1, 40)):
        data = Dataretreiver(reduce='pca_pure', debug=False, reduction_num_cols=i)
        df = data.combined
        # Initialize the experiment
        experiment = EnhancedTimeSeriesExperiment(
            models=models_pure,
            target_column='price',
            forecast_horizon=192,
            n_splits=3,
            step_size=192 # 8 days are skipped between each forecast.
        )
        # Run the experiments with these custom combinations
        experiment.run_feature_group_experiments(
            df=df,
            add_all_columns=True,
            add_base_columns=False,
        )
        results_df_MAE = experiment.summarize_feature_group_results(metric='MAE')
        results_df_MSE = experiment.summarize_feature_group_results(metric='RMSE')
        results_df_time = experiment.summarize_feature_group_results(metric='elapsed_time')
        print('='*50)
        print(f"Iterataion {i} results for pca_pure")
        print("MAE:")
        print(results_df_MAE)
        print("RMSE")
        print(results_df_MSE)
        print("Elapsed time")
        print(results_df_time)
        print('='*50)
        experiment.save_feature_group_results_to_csv(f"../results/opt_num_areas/pca_pure_{i}.csv")

In [7]:
def pearson():
    print("PEARSON STARTED")
    models_pearson = {
        'BASELINE': Baseline(),
        'LGBM': LGBM(),
        #'ARIMA': Arima(order=[1,0,1], seasonal_order=[1,1,1,24]),
        'NEURALPROPHET': Neuralprophet(),
        'TimeGPT': TimeGPT()

    }
    for i in tqdm(range(1, 40)):
        data = Dataretreiver(reduce='pearson', debug=True, reduction_num_cols=i)
        df = data.combined
        # Initialize the experiment
        experiment = EnhancedTimeSeriesExperiment(
            models=models_pearson,
            target_column='price',
            forecast_horizon=192,
            n_splits=3,
            step_size=192 # 8 days are skipped between each forecast.
        )   
        # Run the experiments with these custom combinations
        experiment.run_feature_group_experiments(
            df=df,
            add_all_columns=True,
            add_base_columns=False,
        )
        results_df_MAE = experiment.summarize_feature_group_results(metric='MAE')
        results_df_MSE = experiment.summarize_feature_group_results(metric='RMSE')
        results_df_time = experiment.summarize_feature_group_results(metric='elapsed_time')
        print('='*50)
        print(f"Iterataion {i} results for pearson")
        print("MAE:")
        print(results_df_MAE)
        print("RMSE")
        print(results_df_MSE)
        print("Elapsed time")
        print(results_df_time)
        print('='*50)
        experiment.save_feature_group_results_to_csv(f"../results/opt_num_areas/pearson_{i}.csv")

In [8]:
kmeans()
pure()
pearson()

PCA_KMEANS STARTED


  0%|          | 0/12 [00:00<?, ?it/s]

Identified area columns: {'sun': {22: 'sun_area_22', 13: 'sun_area_13', 33: 'sun_area_33', 10: 'sun_area_10', 36: 'sun_area_36', 34: 'sun_area_34', 29: 'sun_area_29', 11: 'sun_area_11', 16: 'sun_area_16', 4: 'sun_area_4', 18: 'sun_area_18', 35: 'sun_area_35', 20: 'sun_area_20', 21: 'sun_area_21', 32: 'sun_area_32', 3: 'sun_area_3', 12: 'sun_area_12', 27: 'sun_area_27', 24: 'sun_area_24', 25: 'sun_area_25', 1: 'sun_area_1', 19: 'sun_area_19', 9: 'sun_area_9', 17: 'sun_area_17', 23: 'sun_area_23', 26: 'sun_area_26', 40: 'sun_area_40', 2: 'sun_area_2'}, 'wind': {22: 'wind_area_22', 9: 'wind_area_9', 34: 'wind_area_34', 13: 'wind_area_13', 26: 'wind_area_26', 11: 'wind_area_11', 32: 'wind_area_32', 15: 'wind_area_15', 25: 'wind_area_25', 35: 'wind_area_35', 7: 'wind_area_7', 4: 'wind_area_4', 1: 'wind_area_1', 23: 'wind_area_23', 36: 'wind_area_36', 33: 'wind_area_33', 16: 'wind_area_16', 29: 'wind_area_29', 5: 'wind_area_5', 2: 'wind_area_2', 17: 'wind_area_17', 20: 'wind_area_20', 30: 'w

  0%|          | 0/20 [00:00<?, ?it/s]

`Forecaster` refitted using the best-found lags and parameters, and the whole data set: 
  Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48] 
  Parameters: {'n_estimators': 800, 'max_depth': 3, 'min_data_in_leaf': 71, 'learning_rate': 0.32631417602951285, 'feature_fraction': 0.9914051578268095, 'max_bin': 250, 'reg_alpha': 0.9756497942847593, 'reg_lambda': 0.3311404017818648}
  Backtesting metric: 206.20878288174703


...

Finding best initial lr:   0%|          | 0/241 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 65it [00:00, ?it/s]

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Inferred freq: h
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Querying model metadata...
INFO:nixtla.nixtla_client:Using historical exogenous features: ['sun_area_6', 'sun_area_11', 'sun_area_1', 'sun_area_16', 'sun_area_7', 'sun_area_2', 'sun_area_3', 'sun_area_21', 'sun_area_8', 'sun_area_26', 'sun_area_4', 'sun_area_12', 'sun_area_13', 'sun_area_5', 'sun_area_9', 'sun_area_31', 'sun_area_14', 'sun_area_22', 'sun_area_17', 'sun_area_10', 'sun_area_15', 'sun_area_36', 'sun_area_20', 'sun_area_19', 'sun_area_27', 'sun_area_23', 'sun_area_18', 'sun_area_28', 'sun_area_29', 'sun_area_24', 'sun_area_25', 'sun_area_32', 'sun_area_30', 'sun_area_37', 'sun_area_33', 'sun_area_38', 'sun_area_34', 'sun_area_35', 'sun_area_39', 'wind_area_12', 'wind_area_13', 'wind_area_17', 'wind_area_16', 'wind_area_18', 'wind_area_27', 'wind_area_7', 'wind_area_11', 'wind_area_6', 'wind_area_22', 'wind

Iterataion 39 results for pearson
MAE:
                     BASELINE                LGBM       NEURALPROPHET  \
all_areas  243.1759 ± 39.2822  181.3250 ± 31.0604  265.1355 ± 84.8096   

                       TimeGPT  
all_areas  474.8940 ± 172.9528  
RMSE
                     BASELINE                LGBM       NEURALPROPHET  \
all_areas  320.1209 ± 56.7897  230.0921 ± 40.2677  321.0259 ± 97.5341   

                       TimeGPT  
all_areas  534.8889 ± 162.6442  
Elapsed time
                  BASELINE              LGBM       NEURALPROPHET  \
all_areas  0.0120 ± 0.0042  86.0131 ± 2.1155  193.7989 ± 17.6633   

                   TimeGPT  
all_areas  5.5215 ± 0.2747  
Saved feature group results to ../results/opt_num_areas/pearson_39_20250516_082347.csv
